# Test Query
Purpose is to perform a query against the index created.
This is only meant to test the retrieval portion of the RAG application.

## Required for this step
- Azure AI Search Service



In [1]:
# # Import required libraries  
import os  
import json
from azure.core.credentials import AzureKeyCredential  
from azure.search.documents import SearchClient  
from azure.search.documents.indexes import SearchIndexClient, SearchIndexerClient  
from azure.search.documents.models import (
    QueryType,
    VectorizedQuery,
)
from openai import AzureOpenAI
from tenacity import retry, wait_random_exponential, stop_after_attempt 


In [2]:
#Load the configuration details for the Cognitive Search Service and Azure OpenAI Instance
#Credentials should be secured using a more secure method such as Azure KeyVault
from dotenv import load_dotenv
load_dotenv()

# Load configuration from environment variables
config = { 
    "search_service_name": os.getenv("SEARCH_SERVICE_NAME"),
    "search_admin_key": os.getenv("SEARCH_ADMIN_KEY"),
    "search_index_name": os.getenv("SEARCH_INDEX_NAME"),
    "search_index_schema_file": os.getenv("SEARCH_INDEX_SCHEMA_FILE"),
    "search_api_version": os.getenv("SEARCH_API_VERSION", "2023-07-01"),
    "data_root_dir": os.getenv("DATA_ROOT_DIR", "./data"),
    "openai_api_base": os.getenv("OPENAI_API_BASE"),
    "openai_api_key": os.getenv("OPENAI_API_KEY"),
    "openai_api_version": os.getenv("OPENAI_API_VERSION", "2023-05-15"),
    "openai_embedding_model": os.getenv("OPENAI_EMBEDDING_MODEL", "text-embedding-3-large")
}


# Azure AI Search Config
search_service_name = config["search_service_name"]
search_service_url = "https://{}.search.windows.net/".format(search_service_name)
search_admin_key = config["search_admin_key"]
index_name = config["search_index_name"]
index_schema_file = config["search_index_schema_file"]
search_api_version = config["search_api_version"]
search_headers = {  
    'Content-Type': 'application/json',  
    'api-key': search_admin_key  
}  

index_client = SearchIndexClient(
        endpoint=search_service_url, credential=AzureKeyCredential(search_admin_key))
search_client = SearchClient(endpoint=search_service_url, index_name=index_name, credential=AzureKeyCredential(search_admin_key))

print ('Search Service Name:', search_service_name)
print ('Index Name:', index_name)

#Azure OpenAI
api_base = config["openai_api_base"]
api_key = config["openai_api_key"]
openai_api_version = config["openai_api_version"]
embedding_model = config["openai_embedding_model"]

# gets the API Key from environment variable AZURE_OPENAI_API_KEY
client = AzureOpenAI(
    api_version=openai_api_version,
    azure_endpoint=api_base,
    api_key=api_key
)


Search Service Name: agent-ai-search-ibnm
Index Name: finance-pro


In [3]:
# Function to generate embeddings for title and content fields, also used for query embeddings
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
def generate_embeddings(text):
    response = client.embeddings.create(
        input=text,
        model=embedding_model
    )
    return json.loads(response.model_dump_json())["data"][0]['embedding']

# generate_embeddings('test')

In [4]:
# Pure Hubrid Search
query = "how do you value a company??"  
  
vector_query = VectorizedQuery(vector=generate_embeddings(query), k_nearest_neighbors=3, fields="vector")
  
results = search_client.search(  
    search_text=query,  
    vector_queries= [vector_query],
    select=["title", "chunk"],
    top=3
)  
  
for result in results:  
    print(f"Title: {result['title']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['chunk']}")  
    print("==============================\n")

Title: Enterprise Value (EV): Comprehensive Company Valuation Metric
Score: 0.03333333507180214
Content: <figure>  
![](figures/95)  
</figure>  
What Does Enterprise Value (EV) Mean? A measure of a company's value that often is used as an alternative to straightforward market capitalization. EV is calculated as market  
· Stock Market  
<!-- PageHeader="THE INVESTOPEDIA GUIDE TO WALL SPEAK 93" -->  
cap plus debt, minority interest, and preferred shares, minus total cash and cash equivalents.

Title: Market Value vs. Book Value: Understanding Investor Perspectives
Score: 0.03083491325378418
Content: (1) In the context of securities, market value is often different from book value because the market value reflects future expectations. Most investors who pick stocks by using fundamental analysis look at a company's market value and then determine whether the mar- ket value is adequate or is undervalued in comparison to its book value, net assets, or another measure.  
RELATED TERMS:  
·

In [5]:
# Pure Hybrd+Semantic Search
query = "how many shares were repurchased in the second quarter of 2022 by Microsoft?"  
  
vector_query = VectorizedQuery(vector=generate_embeddings(query), k_nearest_neighbors=3, fields="vector")
  
results = search_client.search(  
    search_text=query,  
    vector_queries= [vector_query],
    select=["title", "chunk"],
    query_type=QueryType.SEMANTIC, semantic_configuration_name='vector-semantic-configuration',
    top=3
)  
  
for result in results:  
    print(f"Title: {result['title']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['chunk']}")  
    print("==============================\n")

Title: **Succinct Title:**  
"Share Buybacks: How and Why Companies Repurchase Their Own Stock"
Score: 0.03333333507180214
Content: The repurchase of outstanding shares (repurchase) by a company to reduce the number of shares outstanding in the market; compa- nies buy back shares either to increase the value of available shares (reducing supply) or to eliminate threats by shareholders who may be planning a hostile takeover.  
#### Investopedia explains Buyback -  
A buyback is a method for a company to invest in itself. Buybacks reduce the number of shares outstanding in the market, and that increases the proportion of shares the company owns. Buybacks can be carried out in two ways: (1) Shareholders may be given a tender offer by which they have the option to submit (or tender) a por- tion or all of their shares back to the company within a certain time frame and at a premium to the current market price. The premium is compensation for tendering their shares rather than holding on to 